In [117]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from keras import layers
from keras.activations import relu, sigmoid

In [56]:
!pip install -q -U keras-tuner
import keras_tuner as kt

     |████████████████████████████████| 133 kB 7.6 MB/s 


In [57]:
from keras_tuner import RandomSearch

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


(A) Classification Problem

In [122]:
#Import dataset
dataset = pd.read_csv('/content/drive/MyDrive/DL_Krish_Naik/Churn_Modelling.csv')
X = dataset.iloc[:, 3:13] #Takes columns from 3 to 12
y = dataset.iloc[:, 13] #Take 13th column

In [123]:
#Create dummy variables
geography = pd.get_dummies(X['Geography'], drop_first = True)
gender = pd.get_dummies(X['Gender'], drop_first = True)

In [124]:
X = pd.concat([X, geography, gender], axis = 1)

In [125]:
X = X.drop(['Geography','Gender'], axis = 1)

In [126]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

In [127]:
#Feature Scaling
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.fit_transform(X_test)


In [128]:
#Building ANN
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout

In [129]:
classifier = Sequential()

In [130]:
classifier.add(Dense(units = 10, kernel_initializer= 'he_normal', activation = 'relu', input_dim = 11))
classifier.add(Dropout(0.3))
classifier.add(Dense(units = 20, kernel_initializer= 'he_normal', activation = 'relu'))
classifier.add(Dropout(0.4))
classifier.add(Dense(units = 15, kernel_initializer= 'he_normal', activation = 'relu'))
classifier.add(Dropout(0.2))
classifier.add(Dense(units = 1, kernel_initializer= 'glorot_uniform', activation = 'sigmoid'))

In [131]:
classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

In [132]:
classifier.summary()

Model: "sequential_62"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_19 (Dense)            (None, 10)                120       
                                                                 
 dropout (Dropout)           (None, 10)                0         
                                                                 
 dense_20 (Dense)            (None, 20)                220       
                                                                 
 dropout_1 (Dropout)         (None, 20)                0         
                                                                 
 dense_21 (Dense)            (None, 15)                315       
                                                                 
 dropout_2 (Dropout)         (None, 15)                0         
                                                                 
 dense_22 (Dense)            (None, 1)               

In [133]:
model_history = classifier.fit(X_train, y_train, validation_split = 0.33, batch_size = 10, epochs=100)

Epoch 1/100
536/536 [==============================] - 2s 3ms/step - loss: 0.5781 - accuracy: 0.7429 - val_loss: 0.4863 - val_accuracy: 0.7955
Epoch 2/100
536/536 [==============================] - 2s 3ms/step - loss: 0.5120 - accuracy: 0.7925 - val_loss: 0.4769 - val_accuracy: 0.7955
Epoch 3/100
536/536 [==============================] - 1s 3ms/step - loss: 0.4888 - accuracy: 0.7955 - val_loss: 0.4683 - val_accuracy: 0.7955
Epoch 4/100
536/536 [==============================] - 1s 3ms/step - loss: 0.4732 - accuracy: 0.7985 - val_loss: 0.4584 - val_accuracy: 0.7955
Epoch 5/100
536/536 [==============================] - 1s 3ms/step - loss: 0.4674 - accuracy: 0.7992 - val_loss: 0.4503 - val_accuracy: 0.7955
Epoch 6/100
536/536 [==============================] - 1s 3ms/step - loss: 0.4549 - accuracy: 0.8000 - val_loss: 0.4451 - val_accuracy: 0.7959
Epoch 7/100
536/536 [==============================] - 1s 3ms/step - loss: 0.4534 - accuracy: 0.8033 - val_loss: 0.4424 - val_accuracy: 0.7989

In [134]:
y_pred = classifier.predict(X_test)
y_pred = (y_pred > 0.5)
y_pred

array([[False],
       [False],
       [False],
       ...,
       [False],
       [False],
       [False]])

In [135]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)

In [136]:
from sklearn.metrics import accuracy_score
score = accuracy_score(y_pred, y_test)

In [137]:
score

0.8675

In [138]:
cm

array([[1561,   34],
       [ 231,  174]])

In [145]:
#Perform Hyperparameter Optimization

from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV
from keras.layers import Dense, Activation, Embedding, Flatten, LeakyReLU, BatchNormalization

In [146]:
def create_model(layers, activation):
    model = Sequential()
    for i, nodes in enumerate(layers):
        if i==0:
            model.add(Dense(nodes,input_dim=X_train.shape[1]))
            model.add(Activation(activation))
            model.add(Dropout(0.3))
        else:
            model.add(Dense(nodes))
            model.add(Activation(activation))
            model.add(Dropout(0.3))
            
    model.add(Dense(units = 1, kernel_initializer= 'glorot_uniform', activation = 'sigmoid')) # Note: no activation beyond this point
    
    model.compile(optimizer='adam', loss='binary_crossentropy',metrics=['accuracy'])
    return model
  

In [147]:
model = KerasClassifier(build_fn=create_model, verbose=0)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  """Entry point for launching an IPython kernel.


In [148]:
layers = [[20], [40, 20], [45, 30, 15]]
activations = ['sigmoid', 'relu']
param_grid = dict(layers=layers, activation=activations, batch_size = [128, 256], epochs=[30])
grid = GridSearchCV(estimator=model, param_grid=param_grid,cv=5)

In [149]:
grid_result = grid.fit(X_train, y_train)

In [150]:
grid_result.best_score_, grid_result.best_params_

(0.8557499885559082,
 {'activation': 'relu', 'batch_size': 128, 'epochs': 30, 'layers': [40, 20]})

In [157]:
pred_y = grid.predict(X_test)
y_pred = (pred_y > 0.5)
[i for i , x in enumerate(y_pred) if x]

[5,
 9,
 20,
 44,
 50,
 65,
 69,
 70,
 73,
 74,
 76,
 80,
 84,
 88,
 93,
 122,
 128,
 137,
 147,
 149,
 154,
 155,
 182,
 197,
 203,
 207,
 212,
 213,
 229,
 251,
 253,
 273,
 279,
 303,
 308,
 318,
 320,
 323,
 341,
 343,
 353,
 355,
 365,
 367,
 377,
 386,
 388,
 405,
 418,
 432,
 434,
 452,
 454,
 465,
 467,
 473,
 481,
 485,
 490,
 491,
 502,
 503,
 504,
 513,
 560,
 563,
 573,
 576,
 579,
 585,
 591,
 602,
 609,
 613,
 614,
 615,
 616,
 631,
 639,
 654,
 672,
 681,
 685,
 733,
 736,
 758,
 773,
 787,
 795,
 804,
 821,
 827,
 836,
 856,
 860,
 890,
 891,
 892,
 894,
 900,
 923,
 927,
 940,
 941,
 950,
 957,
 978,
 1004,
 1009,
 1023,
 1025,
 1035,
 1048,
 1062,
 1070,
 1089,
 1095,
 1101,
 1104,
 1105,
 1110,
 1117,
 1123,
 1126,
 1128,
 1131,
 1137,
 1138,
 1139,
 1148,
 1159,
 1174,
 1180,
 1194,
 1198,
 1218,
 1248,
 1257,
 1268,
 1275,
 1280,
 1281,
 1284,
 1295,
 1306,
 1316,
 1335,
 1348,
 1357,
 1388,
 1390,
 1402,
 1430,
 1436,
 1444,
 1446,
 1452,
 1465,
 1468,
 1469,
 147

In [158]:
cm = confusion_matrix(y_test, y_pred)
score = accuracy_score(y_pred, y_test)

In [159]:
cm

array([[1560,   35],
       [ 231,  174]])

In [160]:
score

0.867

(B) Regression Problem

In [58]:
df = pd.read_csv('/content/drive/MyDrive/DL_Krish_Naik/Real_Combine.csv')

In [81]:
df

,T,TM,Tm,SLP,H,VV,V,VM,PM 2.5
0,7.4,9.8,4.8,1017.6,93.0,0.5,4.3,9.4,219.720833
1,7.8,12.7,4.4,1018.5,87.0,0.6,4.4,11.1,182.187500
2,6.7,13.4,2.4,1019.4,82.0,0.6,4.8,11.1,154.037500
3,8.6,15.5,3.3,1018.7,72.0,0.8,8.1,20.6,223.208333
4,12.4,20.9,4.4,1017.3,61.0,1.3,8.7,22.2,200.645833
...,...,...,...,...,...,...,...,...,...
1088,18.1,24.0,11.2,1015.4,56.0,1.8,15.9,25.9,288.416667
1089,17.8,25.0,10.7,1015.8,54.0,2.3,9.4,22.2,256.833333
1090,13.9,24.5,11.4,1015.0,95.0,0.6,8.7,14.8,169.000000
1091,16.3,23.0,9.8,1016.9,78.0,1.1,7.4,16.5,186.041667


In [84]:
df.dropna(subset=["PM 2.5"], inplace = True)

In [85]:
df

,T,TM,Tm,SLP,H,VV,V,VM,PM 2.5
0,7.4,9.8,4.8,1017.6,93.0,0.5,4.3,9.4,219.720833
1,7.8,12.7,4.4,1018.5,87.0,0.6,4.4,11.1,182.187500
2,6.7,13.4,2.4,1019.4,82.0,0.6,4.8,11.1,154.037500
3,8.6,15.5,3.3,1018.7,72.0,0.8,8.1,20.6,223.208333
4,12.4,20.9,4.4,1017.3,61.0,1.3,8.7,22.2,200.645833
...,...,...,...,...,...,...,...,...,...
1088,18.1,24.0,11.2,1015.4,56.0,1.8,15.9,25.9,288.416667
1089,17.8,25.0,10.7,1015.8,54.0,2.3,9.4,22.2,256.833333
1090,13.9,24.5,11.4,1015.0,95.0,0.6,8.7,14.8,169.000000
1091,16.3,23.0,9.8,1016.9,78.0,1.1,7.4,16.5,186.041667


In [86]:
X = df.iloc[:,-1]
y = df.iloc[:-1]

In [89]:
X

0       219.720833
1       182.187500
2       154.037500
3       223.208333
4       200.645833
           ...    
1087    284.166667
1088    288.416667
1089    256.833333
1090    169.000000
1091    186.041667
Name: PM 2.5, Length: 1091, dtype: float64

In [90]:
y

,T,TM,Tm,SLP,H,VV,V,VM,PM 2.5
0,7.4,9.8,4.8,1017.6,93.0,0.5,4.3,9.4,219.720833
1,7.8,12.7,4.4,1018.5,87.0,0.6,4.4,11.1,182.187500
2,6.7,13.4,2.4,1019.4,82.0,0.6,4.8,11.1,154.037500
3,8.6,15.5,3.3,1018.7,72.0,0.8,8.1,20.6,223.208333
4,12.4,20.9,4.4,1017.3,61.0,1.3,8.7,22.2,200.645833
...,...,...,...,...,...,...,...,...,...
1087,16.3,23.0,6.6,1016.9,61.0,1.0,9.4,20.6,284.166667
1088,18.1,24.0,11.2,1015.4,56.0,1.8,15.9,25.9,288.416667
1089,17.8,25.0,10.7,1015.8,54.0,2.3,9.4,22.2,256.833333
1090,13.9,24.5,11.4,1015.0,95.0,0.6,8.7,14.8,169.000000


In [108]:
def build_model(hp):
    model = keras.Sequential()
    for i in range(hp.Int('num_layers', 2, 20)):
        model.add(layers.Dense(units=hp.Int('units_' + str(i),
                                            min_value=32,
                                            max_value=512,
                                            step=32),
                               activation='relu'))
    model.add(layers.Dense(1, activation='linear'))
    model.compile(
        optimizer=keras.optimizers.Adam(
            hp.Choice('learning_rate', [1e-2, 1e-3, 1e-4])),
        loss='mean_absolute_error',
        metrics=['mean_absolute_error'])
    return model

In [110]:
tuner = RandomSearch(
    build_model,
    objective='val_mean_absolute_error',
    max_trials=5,
    executions_per_trial = 3,
    directory = 'proj',
    project_name = 'Air Quality Index'
)

In [111]:
tuner.search_space_summary()

Search space summary
Default search space size: 4
num_layers (Int)
{'default': None, 'conditions': [], 'min_value': 2, 'max_value': 20, 'step': 1, 'sampling': None}
units_0 (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 512, 'step': 32, 'sampling': None}
units_1 (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 512, 'step': 32, 'sampling': None}
learning_rate (Choice)
{'default': 0.01, 'conditions': [], 'values': [0.01, 0.001, 0.0001], 'ordered': True}


In [112]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state=0)

In [113]:
tuner.search(X_train, y_train, epochs=5, validation_data=(X_test, y_test))

Trial 5 Complete [00h 00m 19s]
val_mean_absolute_error: 135.95547993977866

Best val_mean_absolute_error So Far: 130.6375528971354
Total elapsed time: 00h 01m 08s
INFO:tensorflow:Oracle triggered exit


In [114]:
tuner.results_summary()

Results summary
Results in proj/Air Quality Index
Showing 10 best trials
Trial summary
Hyperparameters:
num_layers: 12
units_0: 288
units_1: 512
learning_rate: 0.01
units_2: 448
units_3: 320
units_4: 256
units_5: 192
units_6: 288
units_7: 32
units_8: 32
units_9: 32
units_10: 32
units_11: 32
Score: 130.6375528971354
Trial summary
Hyperparameters:
num_layers: 17
units_0: 224
units_1: 480
learning_rate: 0.001
units_2: 224
units_3: 416
units_4: 448
units_5: 512
units_6: 480
units_7: 64
units_8: 448
units_9: 160
units_10: 64
units_11: 480
units_12: 32
units_13: 32
units_14: 32
units_15: 32
units_16: 32
Score: 133.25576782226562
Trial summary
Hyperparameters:
num_layers: 18
units_0: 96
units_1: 480
learning_rate: 0.0001
units_2: 384
units_3: 448
units_4: 96
units_5: 288
units_6: 256
units_7: 320
units_8: 96
units_9: 416
units_10: 160
units_11: 480
units_12: 448
units_13: 192
units_14: 192
units_15: 160
units_16: 320
units_17: 32
Score: 135.95547993977866
Trial summary
Hyperparameters:
num_la